Use lyrics from Irish songs then see how the generated text looks like.

->Building the Word Vocabulary

->Preprocessing the Dataset

->Build, compile and train the Model

->Generating Text

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential

In [ ]:
!gdown --id 15UqmiIm0xwh9mt0IYq2z3jHaauxQSTQT

In [ ]:
data=open('./irish-lyrics-eof.txt').read()
corpus=data.lower().split("\n")
print(corpus)

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words=len(tokenizer.word_index)+1

print(f'word index dictionary: {tokenizer.word_index}')
print(f'total words: {total_words}')

In [8]:
input_sequences=[]
for line in corpus:
  token_list=tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    n_gram_sequence=token_list[:i+1]
    input_sequences.append(n_gram_sequence)

max_sequence_len=max([len(x) for x in input_sequences])
input_sequences=pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre')
xs,labels=input_sequences[:,:-1],input_sequences[:,-1]
ys=tf.keras.utils.to_categorical(labels,num_classes=total_words)

In [ ]:
# Get sample sentence
sentence = corpus[0].split()
print(f'sample sentence: {sentence}')

token_list = []

# Look up the indices of each word and append to the list
for word in sentence: 
  token_list.append(tokenizer.word_index[word])

print(token_list)

In [ ]:
elem_number = 5

# Print token list and phrase
print(f'token list: {xs[elem_number]}')
print(f'decoded to text: {tokenizer.sequences_to_texts([xs[elem_number]])}')

# Print label
print(f'one-hot label: {ys[elem_number]}')
print(f'index of label: {np.argmax(ys[elem_number])}')

In [ ]:
embedding_dim=128
lstm_units=256

model=Sequential([
    Embedding(total_words,embedding_dim,input_length=max_sequence_len-1),
    Bidirectional(LSTM(lstm_units)),
    Dense(total_words,activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

In [ ]:
epochs = 100

history = model.fit(xs, ys, epochs=epochs)

In [20]:
# Define seed text

seed_text = input(print("Enter something to start your poem"))


# Define total words to predict
next_words = 100

# Loop until desired length is reached
for _ in range(next_words):

	# Convert the seed text to a token sequence
	token_list = tokenizer.texts_to_sequences([seed_text])[0]

	# Pad the sequence
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	
	# Feed to the model and get the probabilities for each index
	probabilities = model.predict(token_list, verbose=0)

	# Get the index with the highest probability
	predicted = np.argmax(probabilities, axis=-1)[0]

	# Ignore if index is 0 because that is just the padding.
	if predicted != 0:
		
		# Look up the word associated with the index. 
		output_word = tokenizer.index_word[predicted]

		# Combine with the seed text
		seed_text += " " + output_word

# Print the result	
print(seed_text)

Enter something to start your poem
NoneI am Keshav and I like to fuck men
I am Keshav and I like to fuck men my barley and fair as the wind and they bore your heart in the again loud and ground more they they did be seen from sea and fathers of wind and bells did be reap those wind open sun ones in art ones spend buy they bubblin wind in art drawn open immortal little open dublins mullingar ones wind and fair old ones wind buy turn the rocky street in the night o they night they take me from the sea young young had best wed gone and young more young fair before fair fair never seen the grey brave
